<div style="text-align: center;"> <h1> FURIA Know Your Fan </h1> </div>

<div style="text-align: center;"> <h3> Todos os pip, ferramentas e bibliotecas utilizadas </h3> </div>

- pip install python-dotenv ipywidgets cryptography

<div style="text-align: center;"> <h3> Chave API Oculta (Arquivo .env) </h3> </div>

In [ ]:
from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()

chave_api = os.getenv('API_KEY')

<div style="text-align: center;"> <h3>Coleta de Dados Básicos e Interesses Relacionados à FURIA</h3> </div>
  
- **Formulário Inicial:** Criar um formulário em Jupyter Notebook (usando `ipywidgets` ou `Streamlit`) para capturar dados pessoais fundamentais: nome, CPF, endereço, data de nascimento, email etc. Validar formato de CPF (biblioteca `python-bcpf` ou validação via regex) e outros campos para evitar erros de digitação.  

- **Interesses e Atividades em e-sports:** Incluir no notebook seções ou perguntas interativas sobre interesses em e-sports, times preferidos (FURIA), jogos mais acompanhados, frequência de eventos assistidos, ingressos ou periféricos adquiridos no último ano. Utilizar `pandas` para estruturar as respostas em um DataFrame. Poder-se-á simular coleta de dados de APIs públicas de eventos (por exemplo, dados de torneios CS:GO) ou pedir ao usuário que importe seu histórico (como um CSV de compras) para preencher esse perfil de maneira realista.  

- **Compras Relacionadas:** Se for aplicável, permitir o upload de extratos simplificados ou listas de compras (como merch de teams e-sports). Usar `pandas` ou `openpyxl` para ler esses arquivos e filtrar itens de interesse (palavras-chave relacionadas a jogos, marcas de e-sports, etc).

In [ ]:
import http.server
import socketserver
import webbrowser
import json
import uuid
import os
import hashlib
import threading
import base64
from datetime import datetime
from cryptography.fernet import Fernet

PORT = 8080
DATA_DIR = "form_data"
KEY_FILE = "rg_encryption.key"

# Carrega ou gera a chave de criptografia
if not os.path.exists(KEY_FILE):
    key = Fernet.generate_key()
    with open(KEY_FILE, 'wb') as kf:
        kf.write(key)
else:
    with open(KEY_FILE, 'rb') as kf:
        key = kf.read()
fernet = Fernet(key)

class MyHandler(http.server.SimpleHTTPRequestHandler):
    def do_POST(self):
        # Só processa a rota '/submit'
        if self.path != '/submit':
            return super().do_GET()

        # Lê o corpo da requisição como JSON
        content_length = int(self.headers.get('Content-Length', 0))
        raw_body = self.rfile.read(content_length).decode('utf-8')
        try:
            dados = json.loads(raw_body)
        except json.JSONDecodeError:
            self.send_error(400, "Bad Request: JSON inválido")
            return

        # Hash do CPF
        cpf_original = dados.get('cpf')
        if cpf_original:
            dados['cpf'] = hashlib.sha256(cpf_original.encode('utf-8')).hexdigest()
        else:
            print("Aviso: 'cpf' não enviado.")

        # Extrai imagens em Base64 do JSON
        rg_base64     = dados.pop('rgImagem_base64', None)
        selfie_base64 = dados.pop('selfieImagem_base64', None)

        # Adiciona timestamp de submissão
        dados['submitted_at'] = datetime.utcnow().isoformat() + 'Z'

        # Gera ID único e prepara diretório
        user_id = str(uuid.uuid4())
        os.makedirs(DATA_DIR, exist_ok=True)

        # Criptografa e salva RG como arquivo .enc
        if rg_base64:
            try:
                rg_bytes = base64.b64decode(rg_base64)
                encrypted = fernet.encrypt(rg_bytes)
                enc_filename = f"{user_id}_rg.enc"
                enc_path = os.path.join(DATA_DIR, enc_filename)
                with open(enc_path, 'wb') as ef:
                    ef.write(encrypted)
                dados['rgImagem_encrypted'] = enc_filename
            except Exception as e:
                print(f"Erro ao criptografar/salvar RG: {e}")

        # Salva selfie como PNG (ou também criptografe se desejar)
        if selfie_base64:
            try:
                selfie_bytes = base64.b64decode(selfie_base64)
                selfie_filename = f"{user_id}_selfie.png"
                selfie_path = os.path.join(DATA_DIR, selfie_filename)
                with open(selfie_path, 'wb') as imgf:
                    imgf.write(selfie_bytes)
                dados['selfieImagem_file'] = selfie_filename
            except Exception as e:
                print(f"Erro ao salvar Selfie: {e}")

        # Grava JSON de metadados de forma atômica
        file_path = os.path.join(DATA_DIR, f"{user_id}.json")
        temp_path = file_path + ".tmp"
        try:
            with open(temp_path, 'w', encoding='utf-8') as f:
                json.dump(dados, f, indent=4, ensure_ascii=False)
            os.replace(temp_path, file_path)
        except Exception as e:
            print(f"Erro ao salvar JSON: {e}")
            self.send_error(500, "Internal Server Error: falha ao salvar dados")
            return

        print(f"Dados salvos (metadados + RG criptografado) em: {DATA_DIR}")

        # Envia resposta de sucesso
        response = {
            'status': 'success',
            'message': 'Dados eviados e criptografados com sucesso!',
            'user_id': user_id
        }
        self.send_response(200)
        self.send_header('Content-Type', 'application/json')
        self.end_headers()
        self.wfile.write(json.dumps(response).encode('utf-8'))

        # Desliga o servidor após 4 segundos
        threading.Timer(4.0, self.server.shutdown).start()


def run_server():
    with socketserver.TCPServer(("", PORT), MyHandler) as httpd:
        print(f"Servidor rodando em http://localhost:{PORT}")
        webbrowser.open(f"http://localhost:{PORT}/Form/form.html")
        httpd.serve_forever()

if __name__ == "__main__":
    run_server()


<div style="text-align: center;"> <h3>Validação de Identidade com Abordagem de IA</h3> </div>

- **Upload de Documentos:** Incluir um widget de upload de arquivos de imagem (RG, CNH ou passaporte) e, opcionalmente, uma selfie do usuário.  
- **OCR e Extração de Dados:** Utilizar uma biblioteca de OCR como `pytesseract` ou `easyocr` para extrair texto do documento. Comparar nome e CPF extraídos com os dados básicos fornecidos para consistência.  
- **Reconhecimento Facial:** Para fortalecer a validação, aplicar uma técnica de reconhecimento facial simples. Usar bibliotecas como `face_recognition` ou `OpenCV` com modelos pré-treinados para detectar rostos na selfie e na foto do documento, e então verificar se pertencem à mesma pessoa (por exemplo, comparando descritores faciais).  
- **Feedback de Validação:** Exibir no notebook resultados da validação (válido/inválido) com base na correspondência de texto e face. Fornecer mensagens orientativas caso haja inconsistências (ex: “CPF não confere com o documento enviado”).

### Instale as bibliotecas necessárias

- pytesseract (requer Tesseract OCR engine instalado separadamente)
 
- Pillow (dependência do pytesseract)
 
- face_recognition (requer dlib, pode ser complexo instalar em alguns sistemas)

- ipywidgets (para o widget de upload)

<br>

```pip install pytesseract Pillow face_recognition ipywidgets ```


OCR, Extração e Comparação de Dados Textuais

Definimos uma função para realizar o OCR na imagem do documento completo, extrair o texto, tentar encontrar Nome e CPF e compará-los com dados fornecidos (vamos simular esses dados fornecidos para o exemplo).

In [ ]:
%pip install tesseract pytesseract

In [ ]:
import json
import io
import os
from PIL import Image, ImageFilter, ImageOps
import pytesseract
import re
import hashlib
from cryptography.fernet import Fernet

os.environ["TESSDATA_PREFIX"] = R"C:\Program Files\Tesseract-OCR\tessdata"
pytesseract.pytesseract.tesseract_cmd = R"C:\Program Files\Tesseract-OCR\tesseract.exe"


DATA_DIR = "form_data"
KEY_FILE = "rg_encryption.key"

# --- Carrega chave de criptografia ---
with open(KEY_FILE, "rb") as f:
    key = f.read()
fernet = Fernet(key)

# --- Recupera o último user_id automaticamente ---
json_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".json")]
if not json_files:
    raise FileNotFoundError("Nenhum arquivo JSON encontrado em 'form_data'.")

json_files.sort(key=lambda f: os.path.getmtime(os.path.join(DATA_DIR, f)), reverse=True)
latest_json_path = os.path.join(DATA_DIR, json_files[0])
user_id = os.path.splitext(json_files[0])[0]
print(f"[ℹ] Usando user_id: {user_id}")

# --- Lê JSON ---
with open(latest_json_path, "r", encoding="utf-8") as f:
    form_data = json.load(f)

# --- Dados fornecidos pelo usuário ---
provided_data = {
    "nome": form_data.get("nome", ""),
    # CPF já vem como hash SHA-256
    "cpf_hash": form_data.get("cpf", "")
}

# --- Função de pré-processamento da imagem para OCR ---
def preprocess_image(image):
    image = image.convert("L")
    image = ImageOps.invert(image)
    image = image.filter(ImageFilter.MedianFilter())
    image = ImageOps.autocontrast(image)
    # Binarização para reduzir ruído
    image = image.point(lambda x: 0 if x < 128 else 255, '1')
    return image

# --- Lê e descriptografa RG ---
encrypted_rg_filename = form_data.get("rgImagem_encrypted")
img_documento_completo = None

if encrypted_rg_filename:
    encrypted_path = os.path.join(DATA_DIR, encrypted_rg_filename)
    try:
        with open(encrypted_path, "rb") as f:
            encrypted_data = f.read()
        decrypted_data = fernet.decrypt(encrypted_data)
        img_documento_completo = Image.open(io.BytesIO(decrypted_data))
        print("[✔] RG descriptografado e carregado.")
        img_documento_completo = preprocess_image(img_documento_completo)
    except Exception as e:
        print(f"[✘] Erro ao descriptografar RG: {e}")
else:
    print("Campo 'rgImagem_encrypted' ausente no JSON.")

# --- Lê a selfie ---
selfie_filename = form_data.get("selfieImagem_file")
img_selfie = None

if selfie_filename:
    selfie_path = os.path.join(DATA_DIR, selfie_filename)
    try:
        img_selfie = Image.open(selfie_path)
        print("[✔] Selfie carregada.")
    except Exception as e:
        print(f"[✘] Erro ao carregar selfie: {e}")
else:
    print("Campo 'selfieImagem_file' ausente no JSON.")

# --- Limpeza de CPF ---
def clean_cpf(cpf_str):
    return re.sub(r"\D", "", cpf_str) if isinstance(cpf_str, str) else ""

# --- OCR e extração ---
def perform_ocr_and_extract_data(pil_image, provided_user_data):
    results = {
        'extracted_text': None,
        'extracted_cpf': None,
        'cpf_match': False,
        'extracted_name': None,
        'name_match': False,
        'success': False,
        'message': ''
    }
    if pil_image is None:
        results['message'] = 'Imagem do documento não fornecida.'
        return results

    try:
        text = pytesseract.image_to_string(pil_image, lang='por')
        results['extracted_text'] = text

        # Extrai CPF no formato 000.000.000-00
        m_cpf = re.search(r"(\d{3}\.\d{3}\.\d{3}-\d{2})", text)
        if m_cpf:
            cpf_str = m_cpf.group(1)
            results['extracted_cpf'] = cpf_str
            # Converte para hash SHA-256 e compara com o hash armazenado
            cleaned = clean_cpf(cpf_str)
            sha256 = hashlib.sha256(cleaned.encode('utf-8')).hexdigest()
            results['cpf_match'] = (sha256 == provided_user_data['cpf_hash'])

        # Regex mais flexível para nome (com acentos e variações)
        m_name = re.search(r"[Nn]ome(?:\s+Social)?\s*[:\/-]?\s*([A-Za-zÀ-ÿ\s]+)", text)
        if m_name:
            name_extracted = m_name.group(1).strip()
            results['extracted_name'] = name_extracted
            if name_extracted.lower() == provided_user_data['nome'].lower():
                results['name_match'] = True

        results['success'] = True
        results['message'] = 'OCR e comparação concluídos.'
    except Exception as e:
        results['message'] = f'Erro no OCR: {e}'
    return results

# --- Executa OCR ---
ocr_data = perform_ocr_and_extract_data(img_documento_completo, provided_data)
print("\nResultado OCR:")
print(json.dumps(ocr_data, indent=4, ensure_ascii=False))


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


Sem criptografia do RG

Reconhecimento e Comparação Facial

Agora, definimos uma função para lidar com a comparação facial entre a foto do documento e a selfie

Ao baixar o tesseract, lembrar de baixar a lingua portuguesa e adicionar na pasta -> C:\Program Files\Tesseract-OCR\tessdata

"por.traineddata"

In [ ]:
%pip install opencv-python


In [ ]:
import json
import os
import io
from pathlib import Path
from typing import Any, Dict, Optional

import numpy as np
import cv2
from PIL import Image
from cryptography.fernet import Fernet

# Constants
DATA_DIR = Path("form_data")
KEY_FILE = Path("rg_encryption.key")
CASCADE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
FACE_SIZE = (150, 150)
DISTANCE_THRESHOLD = 60.0  # adjustable threshold for face match


def load_fernet_key(key_path: Path) -> Fernet:
    """
    Load Fernet encryption key from file.
    """
    with key_path.open("rb") as f:
        key = f.read()
    return Fernet(key)


def load_latest_form(data_dir: Path) -> Dict[str, Any]:
    """
    Load the most recent JSON form from data_dir.
    """
    json_files = sorted(data_dir.glob("*.json"), key=lambda f: f.stat().st_mtime, reverse=True)
    if not json_files:
        raise FileNotFoundError(f"No JSON files found in {data_dir}")
    with json_files[0].open("r", encoding="utf-8") as f:
        return json.load(f)


def decrypt_image(encrypted_filename: str, fernet: Fernet, data_dir: Path) -> Image.Image:
    """
    Decrypt an image file and return a PIL Image.
    """
    encrypted_path = data_dir / encrypted_filename
    if not encrypted_path or not encrypted_path.exists():
        raise FileNotFoundError(f"Encrypted file not found: {encrypted_path}")
    data = fernet.decrypt(encrypted_path.read_bytes())
    return Image.open(io.BytesIO(data))


def load_plain_image(filename: str, data_dir: Path) -> Image.Image:
    """
    Load a non-encrypted image from disk as PIL Image.
    """
    img_path = data_dir / filename
    if not img_path.exists():
        raise FileNotFoundError(f"Image file not found: {img_path}")
    return Image.open(img_path)


def pil_to_bgr(img: Image.Image) -> np.ndarray:
    """
    Convert PIL Image to OpenCV BGR numpy array.
    """
    return cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2BGR)


def detect_face(image_bgr: np.ndarray, cascade: cv2.CascadeClassifier) -> Optional[np.ndarray]:
    """
    Detect the first face in a BGR image, crop and resize it.
    Returns the grayscale face ROI or None.
    """
    faces = cascade.detectMultiScale(image_bgr, scaleFactor=1.1, minNeighbors=5)
    if len(faces) == 0:
        return None
    x, y, w, h = faces[0]
    face = image_bgr[y:y+h, x:x+w]
    face_resized = cv2.resize(face, FACE_SIZE)
    return cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)


def compare_faces(gray1: np.ndarray, gray2: np.ndarray) -> float:
    """
    Compute mean absolute difference between two grayscale face images.
    """
    diff = cv2.absdiff(gray1, gray2)
    return float(np.mean(diff))


def perform_face_comparison_opencv(
    img1_pil: Image.Image, img2_pil: Image.Image, threshold: float = DISTANCE_THRESHOLD
) -> Dict[str, Any]:
    """
    Compare two PIL images containing faces using OpenCV Haar cascades.
    Returns a dict with match result, distance, and status.
    """
    results: Dict[str, Any] = {
        'face_match': False,
        'distance': None,
        'success': False,
        'message': ''
    }
    try:
        img1_bgr = pil_to_bgr(img1_pil)
        img2_bgr = pil_to_bgr(img2_pil)

        cascade = cv2.CascadeClassifier(CASCADE_PATH)
        if cascade.empty():
            raise RuntimeError(f"Failed to load cascade classifier at {CASCADE_PATH}")

        face1 = detect_face(img1_bgr, cascade)
        face2 = detect_face(img2_bgr, cascade)

        if face1 is None or face2 is None:
            results['message'] = "No face detected in one or both images."
            return results

        distance = compare_faces(face1, face2)
        results['distance'] = float(distance)
        results['face_match'] = bool(distance < threshold)
        results['success'] = True
        results['message'] = "Comparison successful with OpenCV."
    except Exception as e:
        results['message'] = f"Error during face comparison: {e}"
    return results


if __name__ == "__main__":
    # Initialize
    fernet = load_fernet_key(KEY_FILE)
    form_data = load_latest_form(DATA_DIR)

    # Decrypt RG image
    encrypted_rg = form_data.get("rgImagem_encrypted")
    img_doc = decrypt_image(encrypted_rg, fernet, DATA_DIR)

    # Load selfie (not encrypted)
    selfie_filename = form_data.get("selfieImagem_file")
    img_selfie = load_plain_image(selfie_filename, DATA_DIR)

    # Run comparison
    result = perform_face_comparison_opencv(img_doc, img_selfie)

    # Print JSON-safe result
    print(json.dumps(result, indent=4, ensure_ascii=False))


### 5. Feedback de Validação Final Combinado

Esta função recebe os resultados das etapas anteriores e fornece um veredito final e feedback detalhado.

In [ ]:
# -*- coding: utf-8 -*-

def provide_final_validation_feedback(ocr_results, face_results):
    """
    Combina os resultados do OCR/Dados e Reconhecimento Facial para dar o feedback final.

    Args:
        ocr_results (dict): Dicionário com resultados do OCR e comparação de dados.
        face_results (dict): Dicionário com resultados da comparação facial.
    """
    print("--- Resultado da Validação de Identidade ---")

    validation_status = "Inválido"
    messages = []

    # Avaliar resultados do OCR e Dados
    if not ocr_results['success']:
        messages.append(f"Falha no processamento de dados do documento: {ocr_results['message']}")
    else:
        if not ocr_results['name_match']:
            messages.append("O nome extraído do documento NÃO confere com o nome fornecido.")
        if not ocr_results['cpf_match']:
             messages.append("O CPF extraído do documento NÃO confere com o CPF fornecido.")
        if ocr_results['name_match'] and ocr_results['cpf_match']:
             messages.append("Dados de Nome e CPF do documento conferem com os dados fornecidos.")
        elif ocr_results['success'] and not (ocr_results['name_match'] or ocr_results['cpf_match']):
             messages.append("Nenhum dado chave (Nome, CPF) pôde ser confirmado a partir do documento ou não confere com os dados fornecidos.")


    # Avaliar resultados da Comparação Facial
    if not face_results['success']:
        messages.append(f"Falha no processamento facial: {face_results['message']}")
    else:
        if face_results['face_match']:
            messages.append(f"A face na selfie corresponde à face na foto do documento (Distância: {face_results['distance']:.4f}).")
        else:
            messages.append(f"A face na selfie NÃO corresponde à face na foto do documento (Distância: {face_results['distance']:.4f}).")


    # Determinar status final
    # Condição para Válido: OCR processado com sucesso + Dados de Nome E CPF conferem + Comparação facial deu match
    # Note: A lógica de validação pode ser ajustada conforme a regra de negócio.
    # Aqui, exigimos match nos dados essenciais E match facial.
    if ocr_results['success'] and ocr_results['name_match'] and ocr_results['cpf_match'] and face_results['success'] and face_results['face_match']:
         validation_status = "Válido"
         messages.append("Todas as verificações de identidade foram bem-sucedidas.")
    else:
         validation_status = "Inválido"
         messages.append("A validação de identidade falhou devido às inconsistências ou erros acima.")


    # --- Exibir o Feedback ---
    print(f"\nSTATUS FINAL: {validation_status}")
    print("\nDetalhes:")
    for msg in messages:
        print(f"- {msg}")

    print("------------------------------------------")


# --- Executar a etapa de Feedback Final ---
# Certifique-se de que ocr_data_results e face_comparison_results estão definidos
if 'ocr_data_results' in locals() and 'face_comparison_results' in locals():
     provide_final_validation_feedback(ocr_data_results, face_comparison_results)
else:
     print("Resultados das etapas anteriores (OCR e Reconhecimento Facial) não disponíveis.")
     print("Por favor, execute todas as células em ordem.")

<div style="text-align: center;"> <h3>Integração com Redes Sociais (Simulada)</h3> </div>

- **Linkagem de Contas:** Na ausência de integrações reais, simular o processo solicitando ao usuário os nomes de usuário ou perfis em redes sociais (Twitter, Facebook, Instagram). Criar células de código comentadas que demonstram como usar APIs (ex: `tweepy` para Twitter, `facebook-sdk` para Facebook) para conectar as contas.  
- **Extração de Atividades Relacionadas a e-sports:** Para cada rede social simulada, mostrar como coletar dados relevantes: por exemplo, obter os últimos *tweets* do usuário e filtrar menções a e-sports ou FURIA; listar as páginas seguidas no Facebook com temas de gaming; ou verificar hashtags usadas em posts do Instagram. Usar `requests` e `BeautifulSoup` ou clientes de API para simulação. Armazenar essas informações em DataFrames para análise.  
- **Monitoramento de Interações:** Demonstrar código que analisa curtidas, comentários ou retweets em publicações de e-sports (fingindo as permissões de API). Por exemplo, usar `tweepy` para buscar tweets que mencionem FURIA ou eventos de e-sports que o usuário retuitou ou comentou.  

<div style="text-align: center;"> <h3>Enriquecimento de Perfil com Dados Sociais e Multimídia</h3> </div>

- **Análise de Comentários:** Para integrar comentários prévios do usuário no YouTube, Reddit e Twitter, incluir blocos que consumam APIs ou dados locais de análise anterior (supondo que existam). Usar `google-api-python-client` para extrair comentários de vídeos de e-sports do YouTube, `PRAW` para posts/comentários no Reddit, e `tweepy` ou dados simulados para tweets.  
- **Processamento de Linguagem Natural:** Aplicar NLP para entender o perfil do usuário: usar bibliotecas como `transformers` ou `spaCy` para classificar sentimento, identificar tópicos ou palavras-chave frequentes nesses comentários. Por exemplo, gerar um gráfico de palavras-chave mais mencionadas em e-sports, ou uma análise de sentimento geral sobre jogos específicos.  
- **Integração de Informações:** Combinar esses insights com os interesses declarados pelo usuário. Exibir visualmente (via `matplotlib` ou `seaborn`) uma nuvem de palavras ou gráfico que mostre as categorias de e-sports mais relevantes para o perfil (baseado em interesses + análise de comentários).  
- **Perfis em Sites de e-Sports:** Permitir que o usuário insira links para seus perfis em plataformas de e-sports (como GameBattles, HLTV, Liquipedia). Usar `requests` e `BeautifulSoup` para raspar detalhes do perfil (jogos, histórico de partidas). Em seguida, aplicar um modelo de IA (ex: `transformers` BERT) para classificar se o conteúdo textual do perfil é relevante às preferências do usuário (por exemplo, buscando termos de jogos citados pelo usuário). Mostrar se há “match” entre interesses do usuário e informações do perfil scraped.  

<div style="text-align: center;"> <h3>Estruturação do Notebook</h3> </div>

- Organizar o notebook em seções claras conforme as etapas acima: **Coleta de Dados**, **Validação de Identidade**, **Integração de Redes Sociais**, **Enriquecimento com Dados Sociais**, **Conclusão**.  
- Incluir explicações breves em cada seção usando células Markdown, resumindo o objetivo daquela etapa. Combinar descrições em texto com células de código demonstrativas.  
- Sugerir bibliotecas específicas no contexto de cada etapa: por exemplo, mencionar `ipywidgets` ou `streamlit` na coleta de dados, `pytesseract`/`OpenCV` na validação de documentos, `tweepy`/`PRAW`/`BeautifulSoup` na integração social, e `transformers`/`spaCy` na análise de linguagem.

<div style="text-align: center;"> <h3>Dicas de Apresentação do Protótipo</h3> </div>

- **Formatação Atraente:** Usar cabeçalhos (`#`, `##`), listas e imagens (logotipos de e-sports, ícones de redes sociais) para tornar o notebook visualmente agradável. Células Markdown bem elaboradas ajudam na legibilidade.  
- **Interatividade:** Incluir elementos interativos (sliders, botões de upload, caixas de seleção) via `ipywidgets` para simular um fluxo real de uso. Isso torna a demonstração dinâmica mesmo no ambiente de notebook.  
- **Visualização de Dados:** Aproveitar gráficos (matplotlib, seaborn ou plotly) para mostrar perfis de interesse ou resultados das análises de comentários. Um gráfico de barras ou nuvem de palavras torna o conteúdo mais didático.  
- **Narração do Código:** Inserir comentários explicativos e outputs de exemplo que guiem o avaliador pelo processo passo a passo. Ao final, apresentar um breve resumo dos resultados obtidos para evidenciar que todos os requisitos foram atendidos.  

<div style="text-align: center;"> <h3>Conclusão</h3> </div>

Este plano garante uma implementação completa dos requisitos do desafio, integrando coleta de informações pessoais e de interesse em e-sports, validação de identidade baseada em IA, simulação de integração social e enriquecimento de perfil com dados externos. A organização em seções claras, o uso de bibliotecas especializadas (ex: **Streamlit/ipywidgets** para interfaces, **OpenCV/face_recognition** para validação, **transformers/spaCy** para IA, **pandas** para dados) e as sugestões de apresentação asseguram uma entrega alinhada e de fácil acompanhamento, mesmo no formato de notebook.